# SimilarWebの数値取得

## ライブラリの呼び出し

In [1]:
import sys,json,os
import urllib.request
import pandas as pd
from dotenv import load_dotenv, find_dotenv

## 環境変数の呼び出し

In [2]:
#  ディレクトリから.envファイルを探す
dotenv_path = find_dotenv()

# 環境変数をロード
load_dotenv(dotenv_path)

# API_KEYに環境変数を代入
API_KEY = os.environ.get("SimilarWeb_API_KEY")

## APIコール用のURL作成

In [3]:
version = 'v1'
domain = 'employment.en-japan.com'
endpoint_group = 'traffic-and-engagement'
endpoint = 'visits'
start_date = '2017-09'
end_date = '2017-11'

url_format = 'https://api.similarweb.com/{}/website/{}/{}/{}?api_key={}&start_date={}&end_date={}&main_domain_only=false&granularity=monthly&country=JP'
url = url_format.format(version,domain,endpoint_group,endpoint,API_KEY,start_date,end_date)

## レスポンスを取得

In [4]:
# APIコールし、HTTPレスポンスをresponseに格納
response = urllib.request.urlopen(url)

# HTTPレスポンスをjsonにパース
read = response.read()
data = json.loads(read)

# APIコールが失敗した場合はEmptyを返す
if data['meta']['status'] != 'Success':
    print('Empty')

In [5]:
data

{'meta': {'device': 'Desktop',
  'last_updated': '2017-12-31',
  'request': {'country': 'JP',
   'domain': 'employment.en-japan.com',
   'end_date': '2017-11-30',
   'format': None,
   'granularity': 'Monthly',
   'limit': None,
   'main_domain_only': False,
   'start_date': '2017-09-01'},
  'status': 'Success'},
 'visits': [{'date': '2017-09-01', 'visits': 2559298.422281931},
  {'date': '2017-10-01', 'visits': 2935992.3870797404},
  {'date': '2017-11-01', 'visits': 2451343.8759825565}]}

In [6]:
print('デバイス: ' + data['meta']['device'])
print('ドメイン: ' + data['meta']['request']['domain'])
print('取得幅: ' + data['meta']['request']['granularity'])
print('期間: ' + data['visits'][0]['date'])
print('値: ' + str(data['visits'][0]['visits']))

デバイス: Desktop
ドメイン: employment.en-japan.com
取得幅: Monthly
期間: 2017-09-01
値: 2559298.422281931


In [7]:
# カテゴリの格納
device = data['meta']['device']
domain = data['meta']['request']['domain']
granularity = data['meta']['request']['granularity']

In [8]:
metrics = 'visits'
# 結果格納用のリスト
meta = data['meta']
result = []

# 結果の格納
#for visit in data['visits']:
for visit in data[metrics]:
    
    # 期間ごとにリストに格納
    single = [device, domain, granularity]
    for val in visit.values():
        
        single.append(val)

    # 期間ごとのリストを結果に格納
    result.append(single)

In [9]:
result

[['Desktop',
  'employment.en-japan.com',
  'Monthly',
  '2017-09-01',
  2559298.422281931],
 ['Desktop',
  'employment.en-japan.com',
  'Monthly',
  '2017-10-01',
  2935992.3870797404],
 ['Desktop',
  'employment.en-japan.com',
  'Monthly',
  '2017-11-01',
  2451343.8759825565]]

In [10]:
dfvi = pd.DataFrame(result)

In [11]:
dfvi

,0,1,2,3,4
0,Desktop,employment.en-japan.com,Monthly,2017-09-01,2.559298e+06
1,Desktop,employment.en-japan.com,Monthly,2017-10-01,2.935992e+06
2,Desktop,employment.en-japan.com,Monthly,2017-11-01,2.451344e+06


In [12]:
def get_call_url(version,domain,endpoint_group,endpoint,API_KEY,start_date,end_date):
    url_format = 'https://api.similarweb.com/{}/website/{}/{}/{}?api_key={}&start_date={}&end_date={}&main_domain_only=false&granularity=monthly&country=JP'
    url = url_format.format(version,domain,endpoint_group,endpoint,API_KEY,start_date,end_date)
    return(url)

In [13]:
def get_api_data(url):
    # APIコールし、HTTPレスポンスをresponseに格納
    response = urllib.request.urlopen(url)

    # HTTPレスポンスをjsonにパース
    read = response.read()
    data = json.loads(read)

    # APIコールが失敗した場合はEmptyを返す
    if data['meta']['status'] != 'Success':
        return('Empty')
    return(data)

In [14]:
def get_index_data(data):
    # 結果格納用のリスト
    meta = data['meta']
    index = []

    # 結果の格納
    for visit in data['visits']:

        # 期間ごとにリストに格納
        single = [device, domain, granularity,visit['date']]

        # 期間ごとのリストを結果に格納
        index.append(single)
        
    return(index)

In [20]:
version = 'v1'
domain = 'employment.en-japan.com'
endpoint_group = 'traffic-and-engagement'
endpoint = 'visits'
start_date = '2017-09'
end_date = '2017-11'

url = get_call_url(version,domain,endpoint_group,endpoint,API_KEY,start_date,end_date)
data = get_api_data(url)
index = get_index_data(data)

df_index = pd.DataFrame(index,columns=['デバイス','ドメイン','期間の幅','期間'])

In [21]:
df_index

,デバイス,ドメイン,期間の幅,期間
0,Desktop,employment.en-japan.com,Monthly,2017-09-01
1,Desktop,employment.en-japan.com,Monthly,2017-10-01
2,Desktop,employment.en-japan.com,Monthly,2017-11-01


In [17]:
def get_metrics(data,metrics):
    # 結果格納用のリスト
    meta = data['meta']
    result = []

    # 結果の格納
    for metric in data[metrics]:

        # 期間ごとにリストに格納
        single = []
        for val in metric.values():

            single.append(val)

        # 期間ごとのリストを結果に格納
        result.append(single)
    return(result)

In [22]:
metrics = 'visits'
metric = pd.DataFrame(get_metrics(data,metrics),columns=['期間','訪問数'])
metric

,期間,訪問数
0,2017-09-01,2.559298e+06
1,2017-10-01,2.935992e+06
2,2017-11-01,2.451344e+06


In [23]:
pd.merge(df_index, metric, on='期間')

,デバイス,ドメイン,期間の幅,期間,訪問数
0,Desktop,employment.en-japan.com,Monthly,2017-09-01,2.559298e+06
1,Desktop,employment.en-japan.com,Monthly,2017-10-01,2.935992e+06
2,Desktop,employment.en-japan.com,Monthly,2017-11-01,2.451344e+06
